In [66]:
import pandas as pd
import re

In [67]:
#some initial data cleaning work
def initial_cleaning(df):
    df.fillna('unknown', inplace=True)
    df.drop_duplicates(inplace=True)

In [68]:
#a tool function for cleaning repeated ce;;s
def apply_repeated(x):
    middle = len(x)//2
    if x != 'unknown':
        return x[:middle + 1]
    else:
        return 'unknown'

In [69]:
#modify the location and area column
def location_area(df,location_index):
    df[['location','area']] = df[location_index].str.split('area:', expand = True)
    df.drop(location_index, axis = 1, inplace = True)
    df['area'].fillna('unknown', inplace = True)
    df['location'] = df['location'].apply(lambda x: x[9:])
    df['location'] = df['location'].apply(apply_repeated)
    df['area'] = df['area'].str.split(',', expand = True)[0]
    df['area'] = df['area'].apply(apply_repeated)

In [118]:
#modify the classification column
def classification(df, filename, classification_index, duplicate_index):
    if filename == 'NZ_Banking_JOBS.xlsx':
        df.drop(duplicate_index, axis = 1, inplace = True)
    df[['classification','Classification']] = df[classification_index].str.split('Classification:', expand = True)
    df['Classification'].fillna('unknown', inplace = True)
    df['Classification'] = df['Classification'].apply(apply_repeated)
    df['classification'] = df['classification'].apply(lambda x: x[15:] if 'classification' in x else 'unknown')
    df[['classification1','classification2','classification3']] = df['classification'].str.split('&', expand = True).fillna('unknown')

In [329]:
#deal with the salary column
def salary(df, filename, classification_index):
    import numpy as np
    df['salary'] = df[classification_index].apply(lambda x: 'unknown' if 'classification' in x else x)
    df.drop('classification', axis = 1, inplace = True)
    if filename == 'NZ_Admin_JOBS.xlsx':
        df['salary'] = df[df['salary'].str.contains('\d',na = False)]['salary'].str.replace('5k','5000', regex = True)
        df['salary'] = df[df['salary'].str.contains('\d',na = False)]['salary'].str.replace('0k','0000', regex = True)
    df['salary'] = df[df['salary'].str.contains('\d', na = False)]['salary'].str.replace(' to ', '-', regex = True)
    df['salary'] = df[df['salary'].str.contains('\d', na = False)]['salary'].str.replace(',','')
    df['salary'].fillna('unknown', inplace = True)
    df[['min_salary','max_salary']] = df['salary'].str.split('-', expand = True)
    #deal with min_salary and max_salary
    df['min_salary'] = df['min_salary'].str.replace(' ','')
    df['max_salary'] = df['max_salary'].str.replace(' ', '')
    df['min_salary'] = df[df['min_salary'].str.contains('\d', na = False)]['min_salary'].str.extract('(\d+.?\d+)')
    df['max_salary'] = df[df['max_salary'].str.contains('\d', na = False)]['max_salary'].str.extract('(\d+.?\d+)')
    df['min_salary'].fillna(0, inplace = True)
    df['max_salary'].fillna(0, inplace = True)
    for r, row in enumerate(df['min_salary'].values):
        if row == 0:
            df.loc[r,'min_salary'] = np.random.randint(45000, 75000, size = 1)[0]
    for r, row in enumerate(df['max_salary'].values):
        if row == 0:
            df.loc[r,'max_salary'] = np.random.randint(45000, 75000, size = 1)[0]
    df['min_salary'] = df['min_salary'].astype('float')
    df['max_salary'] = df['max_salary'].astype('float')
    df.loc[df['min_salary'] < 200, 'min_salary'] = df['min_salary']*8*200
    df.loc[df['max_salary'] < 200, 'max_salary'] = df['max_salary']*8*200

In [330]:
def clean_nz_seek(filename, location_index, classification_index, posted_time_index, duplicate_index):
    df = pd.read_excel(filename, header= None)
    initial_cleaning(df)
    location_area(df, location_index)
    classification(df, filename, classification_index, duplicate_index)
    salary(df, filename, classification_index)
    df.rename(columns={0:'Job Title', 1:'from website', 2:'company name', posted_time_index: 'posted_time'}, inplace = True)
    return df

In [331]:
df = clean_nz_seek('NZ_Banking_JOBS.xlsx', 
              location_index = 3,
              classification_index = 6, 
              posted_time_index = 5,
              duplicate_index = 4)

In [333]:
df.loc[df['min_salary'] == 0] #3884?

,Job Title,from website,company name,posted_time,6,location,area,Classification,classification1,classification2,classification3,salary,min_salary,max_salary
3884,Regulatory and Compliance Manager-12 month con...,https://www.seek.co.nz/job/50575511?type=stand...,at Debbie Graham & Associates Limited,"12d ago,at",classification: Banking & Financial ServicesBa...,Auckland,Auckland Central,Compliance & Risk,Banking,Financial ServicesBanking,Financial Servicessub,unknown,0.0,0.0
